# **Fake EHR Data Analysis**

**Import the libraries needed in this notebook**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

In [ ]:
#Import the csv files needed
conditions=pd.read_csv(r"Tables\conditions.csv")
allergies = pd.read_csv(r"Tables\allergies.csv")
patients=pd.read_csv(r"Tables\patients.csv")
observations=pd.read_csv(r"Tables\observations.csv")
encounters=pd.read_csv(r"Tables\encounters.csv")

## **2. Count Table of patients, cities, counties, diseases, allergies and medications**

In [ ]:
#Create a dataframe containing only disorders using conditions csv table
conditions_new=[]
for i in range(len(conditions)):
    # Check the condition: if the value in the 'x' column is 3
    x = conditions.loc[i,"DESCRIPTION"]
    if x[-9:-1] == "disorder":
        # Append the entire row (as a Series) to the list
        conditions_new.append(conditions.iloc[i])

conditions_new = pd.DataFrame(conditions_new)

In [ ]:
#count of differnt values of our data

num_patients = len(patients["Id"].unique())
num_cities = len(patients["CITY"].unique())
num_counties = len(patients["COUNTY"].unique())
num_female = len(patients[patients["GENDER"]=="F"])
num_male = len(patients[patients["GENDER"]=="M"])
num_cond = len(conditions_new["DESCRIPTION"].unique())
num_allergies = len(allergies[allergies['TYPE'] == 'allergy']['DESCRIPTION'].unique())

d = {'Variable': ["Patient", "Female","Male","City","County","Disorders","Allergies"], 'Total Number': [num_patients,num_female,num_male,num_cities,num_counties,num_cond,num_allergies]}
df = pd.DataFrame(data=d)
df

## **3.	Age distribution with living status**

In [ ]:
#Calculate age of patients
#Copy the original dataframe
patients_new=patients.copy()

#Get today's date
datetoday=datetime.datetime.now()

#Convert BIRTHDATE string to a datetime object
patients_new["BIRTHDATE"]=pd.to_datetime(patients_new['BIRTHDATE'])
patients_new["DEATHDATE"]=pd.to_datetime(patients_new['DEATHDATE'])


#This for loop create two column "Living Status" and "Age"
for i in range(len(patients_new)):

    if pd.isnull(patients_new.loc[i, "DEATHDATE"]): #Check is Deathdate exists or not

        patients_new.loc[i, "Living Status"] = "Alive" #If not exists Alive will be entered

        patients_new.loc[i, "Age"] = (datetoday - patients_new.loc[i, "BIRTHDATE"]).days // 365 #Bcs the patient is alive age will be calculated

    else:

        patients_new.loc[i, "Living Status"] = "Dead" #Else enter Dead

        patients_new.loc[i, "Age"] = ((patients_new.loc[i, "DEATHDATE"]) - patients_new.loc[i, "BIRTHDATE"]).days // 365

#Calculate the mean age of the dead and alive patients
dead_patients_mean_age = int(patients_new[patients_new["Living Status"]=="Dead"]["Age"].mean())
alive_patients_mean_age = int(patients_new[patients_new["Living Status"]=="Alive"]["Age"].mean())


custom_palette = {
    "Alive": "#03DB52", 
    "Dead": "#FE6244", 
}

#Histopgram of age distribution of patients with living status
g=sns.displot(
    data=patients_new,
    x="Age",
    hue="Living Status",
    palette=custom_palette
)
g.set(title="Age Distribution with Living Status"
                            ,xlabel="Age (year)"
                            ,ylabel="Patients")

# Access the underlying Axes object
ax = g.ax

# Add a point 
ax.scatter(dead_patients_mean_age, 10, color="red", s=100, marker="o", zorder=10)
ax.scatter(alive_patients_mean_age, 50, color="green", s=100, marker="o", zorder=10)

# Add text with a background box
ax.text(
    49, 9,fr"Mean Death Age = {dead_patients_mean_age:.1f}",
    color="red",
    fontsize=10,
    fontweight="bold",
    bbox=dict(facecolor="white", edgecolor="red", boxstyle="round,pad=0.3")
)

ax.text(
    55, 48,fr"Mean Living Age = {alive_patients_mean_age:.1f}",
    color="green",
    fontsize=10,
    fontweight="bold",
    bbox=dict(facecolor="white", edgecolor="green", boxstyle="round,pad=0.3")
)


plt.show()




g.savefig("Plots/Age Distribution with Living Status.jpg",format="jpg",dpi=1200)
g.savefig("Plots/Age Distribution with Living Status.pdf",format="pdf")

## **4.	Distribution of patients in top 5 cities with gender**

In [ ]:
#Gender distribution in top 5 most populated cities
# Get the number of patients in each city and convert to df
patients_each_city_df = patients['CITY'].value_counts().to_frame().reset_index()

#Get patients data from top 5 cities
city_patients = patients[patients['CITY'].isin(patients_each_city_df.iloc[0:5,0])]


custom_palette = {
    "F": "#EE4654",        
    "M": "#83DDE3",   
}

# Visualize the number of patients by the most 5 popular city and gender
ax1 = sns.countplot(city_patients,
                     x="CITY", 
                     hue="GENDER",
                     palette= custom_palette)
ax1.set_title("Patients Distribution in Top 5 Cities with Gender")
ax1.set(title="Patients Distribution in Top 5 Cities with Gender"
                            ,xlabel="City"
                            ,ylabel="Patient")


plt.tight_layout()
ax1.figure.savefig("Plots/Patients Distribution in top 5 cities with gender.jpg",format="jpg",dpi=1200)
ax1.figure.savefig("Plots/Patients Distribution in top 5 cities with gender.pdf",format="pdf",bbox_inches="tight")

## **5. What is the distribution of the number of conditions per patient?**

In [ ]:
#merge patients and conditions
patients_conditions_merged = pd.merge(patients_new,conditions_new,left_on="Id",right_on="PATIENT",how="inner")

# Count number of conditions per patient
condition_counts = patients_conditions_merged.groupby('Id')['DESCRIPTION'].nunique()

# Get distribution
distribution = condition_counts.value_counts().sort_index()


sns.set(style="whitegrid")

plt.figure(figsize=(8,5))

ax = sns.histplot(condition_counts, bins=range(1, condition_counts.max()+2), discrete=True,color="#00A8C6")

plt.xlabel('Number of conditions per patient', fontsize=12)
plt.ylabel('Number of patients', fontsize=12)
plt.title('Distribution of Number of Conditions per Patient', fontsize=14)

ax.scatter(3, distribution.max(), color="red", s=100, marker="o", zorder=10)

# Add text with a background box
ax.text(3.5, distribution.max(), 
        fr"{distribution.max():.1f} patients have 3 disorders", 
        color="red", fontsize=12, fontweight="bold",
        bbox=dict(facecolor="white", edgecolor="red", boxstyle="round,pad=0.3"))


plt.show()

ax.figure.savefig("Plots/Distribution of Number of Conditions per Pationt.jpg",format="jpg",dpi=1200)
ax.figure.savefig("Plots/Distribution of Number of Conditions per Pationt.pdf",format="pdf",bbox_inches="tight")

## **6. TWO PLOTS for weight and heaight with Scatter plot of BMI values and categories**

In [ ]:
patients_observations_merged=pd.merge(patients, observations, left_on="Id", right_on="PATIENT", how="inner")
patients_observations_merged

patients_clinical_data = patients_observations_merged.copy()
patients_clinical_data.drop(columns=["DRIVERS","PASSPORT","PREFIX","FIRST","LAST","SUFFIX","MAIDEN","ADDRESS","BIRTHPLACE","FIPS","ZIP","LAT","LON","HEALTHCARE_EXPENSES","HEALTHCARE_COVERAGE"],inplace=True)
patients_clinical_data.info()

patients_clinical_data.drop(columns=["PATIENT","ENCOUNTER"],inplace=True)


# Loop through each unique description
for desc in patients_clinical_data["DESCRIPTION"].unique():
    # Create a valid column name (remove spaces etc.)
    col_name = desc.replace(" ", "_")
    
    # Create the new column and assign values where DESCRIPTION matches
    patients_clinical_data[col_name] = None
    patients_clinical_data.loc[patients_clinical_data["DESCRIPTION"] == desc, col_name] = patients_clinical_data["VALUE"]

#Group by Id and drop the null values
patients_clinical_data = patients_clinical_data.groupby(["Id"], as_index=False).agg(lambda x: x.dropna().iloc[0] if x.dropna().any() else None)

#drop all the unnecessary columns
patients_clinical_data=patients_clinical_data[["Id","CITY","COUNTY","STATE","Body_mass_index_(BMI)_[Ratio]","Body_Weight",
                        "Body_Height","Diastolic_Blood_Pressure","Systolic_Blood_Pressure",
                        "Heart_rate","Respiratory_rate","Tobacco_smoking_status","Stress_level","Glucose_[Mass/volume]_in_Blood",
                        "Oxygen_saturation_in_Arterial_blood",
                        "Mean_blood_pressure",
                        "Hemoglobin"
                        ]]
patients_clinical_data

In [ ]:
#Rename the column
patients_clinical_data.rename(columns={"Body_mass_index_(BMI)_[Ratio]":"BMI"},inplace=True)

#Change the type of BMI, weight and height to float
patients_clinical_data['BMI'] = patients_clinical_data['BMI'].astype(float)
patients_clinical_data['Body_Weight'] = patients_clinical_data['Body_Weight'].astype(float)
patients_clinical_data['Body_Height'] = patients_clinical_data['Body_Height'].astype(float)

#Check the type of each column
patients_clinical_data.info()

In [ ]:
#This for loop create category column "BMI_Category" using BMI value
for i in range(len(patients_clinical_data)):

    if patients_clinical_data.loc[i, "BMI"] < 18.5:
        patients_clinical_data.loc[i, "BMI_Category"] = "Underweight"

    elif (patients_clinical_data.loc[i, "BMI"] < 23) and (patients_clinical_data.loc[i, "BMI"] >= 18.5):
        patients_clinical_data.loc[i, "BMI_Category"] = "Healthy"

    elif (patients_clinical_data.loc[i, "BMI"] < 27.5) and (patients_clinical_data.loc[i, "BMI"] >= 23):
        patients_clinical_data.loc[i, "BMI_Category"] = "Overweight"

    else:
        patients_clinical_data.loc[i, "BMI_Category"] = "Obese"


#Change the order of the columns
patients_clinical_data = patients_clinical_data[['Id', 'CITY', 'COUNTY', 'STATE', 'Body_Weight', 'Body_Height', 'BMI','BMI_Category',
       'Diastolic_Blood_Pressure', 'Systolic_Blood_Pressure', 'Heart_rate',
       'Respiratory_rate', 'Tobacco_smoking_status', 'Stress_level',
       'Glucose_[Mass/volume]_in_Blood', 'Oxygen_saturation_in_Arterial_blood',
       'Mean_blood_pressure', 'Hemoglobin']]


diabetes_conditions = conditions[(conditions ["DESCRIPTION"]=="Diabetes mellitus type 2 (disorder)") | (conditions["DESCRIPTION"]=="Prediabetes")]

patients_with_diabetes = pd.merge( diabetes_conditions, patients_clinical_data,left_on="PATIENT",right_on="Id",how="inner")

In [ ]:
# Define your own color palette (matching your categories)
custom_palette = {
    "Obese": "#1A3D64",        # blue
    "Overweight": "#FE6244",   # orange
    "Healthy": "#5FA4A0",      # green
    "Underweight": "#CD2C58"   # red
}

f, axs = plt.subplots(1, 2, figsize=(10,4))

# Plot 1: Scatter plot on the first subplot
ax=sns.scatterplot(data=patients_clinical_data, 
                x="Body_Weight", 
                y="Body_Height", 
                hue="BMI_Category",
                palette=custom_palette,
                ax=axs[0])
# Set title and labels separately
ax.set(
    title="Relationship Between Weight and Height",
    xlabel="Weight (kg)",
    ylabel="Height (cm)"
)

# Edit the legend
ax.legend(title="BMI Classification")


# Plot 2: Histogram on the second subplot
ax1=sns.histplot(data=patients_with_diabetes, 
             x="BMI_Category", 
             hue="DESCRIPTION",
             palette={ "Prediabetes":"#FFDE49",
                       "Diabetes mellitus type 2 (disorder)":"#FF3D3D"},
             multiple="dodge", 
             ax=axs[1],
             legend=True
             )

ax1.set(title="Disorder Distribution over BMI Categories"
                            ,xlabel="BMI Category"
                            ,ylabel="Patients")

f.tight_layout()

f.figure.savefig("Plots/Weight, Height and BMI with the count of diabetes.jpg",format="jpg",dpi=1200)
f.figure.savefig("Plots/Weight, Height and BMI with the count of diabetes.pdf",format="pdf")
plt.show()

## **7.	What is the average age at death of patients? In which cities? What was their medical conditions? (piechart)**

In [ ]:
#merge conditions with the dead patients
conditions_new_merged = pd.merge(patients_new[patients_new["Living Status"] == "Dead"],
                                conditions_new,
                                left_on="Id", right_on="PATIENT",
                                how="inner"
                                )

#create a new dataframe with specific columns
death_patients_info = conditions_new_merged[['Id', 'BIRTHDATE', 'DEATHDATE','GENDER','CITY', 'STATE', 'COUNTY','Living Status', 'Age', 'PATIENT','CODE','DESCRIPTION']]

#Count the disorders of dead patients
death_descriptions=death_patients_info["DESCRIPTION"].value_counts().sort_values(ascending=False).reset_index()

death_descriptions.set_index("DESCRIPTION",inplace=True)

In [ ]:
from matplotlib.patches import ConnectionPatch

# make figure and assign axis objects
fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(12, 5))
fig.subplots_adjust(wspace=0)

values=death_descriptions[0:5]["count"]


#plot pie chart
myexplode = [0.07, 0, 0, 0,0]
wedges, *_=ax3.pie(data=death_descriptions[0:5]
        ,x='count'
        ,labels=death_descriptions.index[0:5]
        ,explode = [0.07, 0, 0, 0,0]
        ,shadow = True
        ,colors=sns.color_palette('Set2')
        ,autopct=lambda x: '{:.0f}'.format(x*values.sum()/100)
        ,textprops={'fontsize':10}
        ,startangle = 285)

ax3.set_title(
    label="Death Distribution By Top 5 Disorders", 
    fontdict={"fontsize":16},
    pad=20
)

# bar chart parameters
city = list(death_patients_info[death_patients_info["DESCRIPTION"]== "Viral sinusitis (disorder)"]["CITY"].value_counts()[0:5].index)

num = list(death_patients_info[death_patients_info["DESCRIPTION"]== "Viral sinusitis (disorder)"]["CITY"].value_counts()[0:5])
bottom = 1
width = .2


for j, (height, label) in enumerate(reversed([*zip(num, city)])):
    bottom -= height
    bc = ax4.bar(0, height, width, bottom=bottom, color='#66C2A5', label=label
                 ,alpha=0.1 + 0.15 * j)
    ax4.bar_label(bc, labels=[f"{height}"], label_type='center')

ax4.set_title('City')
ax4.legend()
ax4.axis('off')
ax4.set_xlim(- 2.5 * width, 2.5 * width)


# use ConnectionPatch to draw lines between the two plots
theta1, theta2 = wedges[0].theta1, wedges[0].theta2
center, r = wedges[0].center, wedges[0].r
bar_height = sum(num)

# draw top connecting line
x = r * np.cos(np.pi / 180 * theta2) + center[0]
y = r * np.sin(np.pi / 180 * theta2) + center[1]
con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax4.transData,
                      xyB=(x, y), coordsB=ax3.transData)
con.set_color("#C2C2C2")
con.set_linewidth(3)
ax4.add_artist(con)

# draw bottom connecting line
x = r * np.cos(np.pi / 180 * theta1) + center[0]
y = r * np.sin(np.pi / 180 * theta1) + center[1]
con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax4.transData,
                      xyB=(x, y), coordsB=ax3.transData)
con.set_color("#C2C2C2")
ax4.add_artist(con)
con.set_linewidth(3)

fig.figure.savefig("Plots/Death Distribution By Top 5 Disorders.png",format="png",dpi=200)
fig.savefig("Plots/Death Distribution By Top 5 Disorders.pdf",format="pdf",dpi=1200)
plt.show()
